In [1]:
import cv2
import numpy as np
import os
import shutil

def read_labels(label_path):
    with open(label_path, 'r') as file:
        labels = file.readlines()
    labels = [line.strip().split() for line in labels]
    return labels

def write_labels(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(label) + '\n')

def transform_labels(labels, transform, img_width, img_height):
    new_labels = []
    for label in labels:
        cls, x, y, w, h = map(float, label)
        if transform == 'mirror':
            x = 1 - x
        elif transform == 'rotate90':
            x, y = 1-y, x
            w, h = h, w
        elif transform == 'rotate90_mirror':
            x, y = y, x
            w, h = h, w
        elif transform == 'rotate180':
            x, y = 1 - x, 1 - y
        elif transform == 'rotate180_mirror':
            x, y = x, 1-y
        elif transform == 'rotate270':
            x, y = y, 1-x
            w, h = h, w
        elif transform == 'rotate270_mirror':
            x, y = 1 - y, 1 - x
            w, h = h, w

        new_labels.append([str(int(cls)), str(x), str(y), str(w), str(h)])
    return new_labels

def augment_image(img, transform):
    if transform == 'mirror':
        return cv2.flip(img, 1)
    elif transform == 'rotate90':
        return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif transform == 'rotate90_mirror':
        return cv2.flip(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE), 1)
    elif transform == 'rotate180':
        return cv2.rotate(img, cv2.ROTATE_180)
    elif transform == 'rotate180_mirror':
        return cv2.flip(cv2.rotate(img, cv2.ROTATE_180), 1)
    elif transform == 'rotate270':
        return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif transform == 'rotate270_mirror':
        return cv2.flip(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE), 1)

def process_images(image_folder, label_folder, output_folder):
    transformations = [
        'mirror', 
        'rotate90', 
        'rotate90_mirror', 
        'rotate180', 
        'rotate180_mirror', 
        'rotate270', 
        'rotate270_mirror'
    ]
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for image_name in os.listdir(image_folder):
        if image_name.endswith('.jpg') or image_name.endswith('.png'):
            img_path = os.path.join(image_folder, image_name)
            img = cv2.imread(img_path)
            img_height, img_width = img.shape[:2]
            label_path = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
            labels = read_labels(label_path)

            # Copy original image and label
            shutil.copy(img_path, os.path.join(output_folder, image_name))
            shutil.copy(label_path, os.path.join(output_folder, os.path.splitext(image_name)[0] + '.txt'))

            for transform in transformations:
                aug_img = augment_image(img, transform)
                if 'rotate90' in transform or 'rotate270' in transform:
                    aug_labels = transform_labels(labels, transform, img_height, img_width)
                else:
                    aug_labels = transform_labels(labels, transform, img_width, img_height)

                aug_img_name = os.path.splitext(image_name)[0] + f'_{transform}' + os.path.splitext(image_name)[1]
                aug_label_name = os.path.splitext(image_name)[0] + f'_{transform}.txt'

                cv2.imwrite(os.path.join(output_folder, aug_img_name), aug_img)
                write_labels(os.path.join(output_folder, aug_label_name), aug_labels)


image_folder = 'Data'+os.sep+'Augmenting'+os.sep+'Raw'
label_folder = 'Data'+os.sep+'Augmenting'+os.sep+'Raw'
output_folder = 'Data'+os.sep+'Augmenting'+os.sep+'Final'

process_images(image_folder, label_folder, output_folder)
